In [308]:
import pandas as pd
import numpy as np
import re, reprlib, sys

%load_ext cython
%load_ext cythonmagic

import nltk as n
import nltk, nltk.classify.util, nltk.metrics, nltk.tokenize, nltk.stem, nltk.book
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.classify import MaxentClassifier
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures as BAM
from nltk.probability import FreqDist, ConditionalFreqDist
from nltk.classify.scikitlearn import SklearnClassifier

from itertools import chain

import sklearn as sk
from sklearn import cross_validation
from sklearn.feature_extraction.text import *
from sklearn.svm import *
from sklearn.pipeline import *
from sklearn.multiclass import *
from sklearn.naive_bayes import *

# n.download()

The cython extension is already loaded. To reload it, use:
  %reload_ext cython
The cythonmagic extension is already loaded. To reload it, use:
  %reload_ext cythonmagic


### Read data and show them

In [309]:
data = pd.read_csv("data.csv", parse_dates=True, infer_datetime_format=True, 
            sep = None, encoding = "latin-1", engine = "python")
data.head()

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180,14861.6,93.29,0.03,15661,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [310]:
# data.describe()

In [311]:
# plit data
train, test = sk.cross_validation.train_test_split(data, train_size = 0.66)

token_dict = {}

# http://billchambers.me/tutorials/2015/01/14/python-nlp-cheatsheet-nltk-scikit-learn.html
# http://glowingpython.blogspot.de/2013/07/combining-scikit-learn-and-ntlk.html
# http://www.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html
# https://stackoverflow.com/questions/10526579/use-scikit-learn-to-classify-into-multiple-categories
# https://github.com/anuraagvak/IRE-PersonalityRecognition-Final/blob/master/ire_report.pdf
# https://github.com/Charudatt89/Personality_Recognition/blob/master/22-9-PersonalityRecognition/Report/Report.pdf

#classif = SklearnClassifier(LinearSVC())
#classif.train(train)

#fdist1 = n.FreqDist(list_of_rows_CD)
#fdist1.most_common(5)
#fdist1.hapaxes()
#set(w.lower() for w in list_of_rows)

### Combine all the strings for each author


In [312]:
combined_dataset = train.groupby(['#AUTHID'])['STATUS'].apply(lambda x: ' '.join(x)).reset_index()
combined_dataset.to_csv(path_or_buf="oneBigString.csv")
combined_dataset.head()

,#AUTHID,STATUS
0,02c37028a782cfda660c7243e45244bb,abcdefghijklmnopqrstuvwxyz qwertyuiopasdfghjkl...
1,03133a828cd0cf52e3752813ce5d818f,Feels aweful right now. Why do you get sick li...
2,03e6c4eca4269c183fa0e1780f73faba,"""I refuse to answer that question on the groun..."
3,06b055f8e2bca96496514891057913c3,is enjoying the cricket...comfy boxers and rai...
4,0724fe854bd455061ba84efecdeff469,Life truly has special ways to show us what we...


### Count how many unique values are there in the first column ?
#### Basically number of users

In [313]:
unique_values = train[train.columns[0]]
pd.Series.nunique(unique_values)

242

In [314]:
# https://stackoverflow.com/questions/21633580/pandas-counting-unique-values-in-a-dataframe
pd.value_counts(unique_values.values.ravel())

e6cdef6f475cce3023c5b715f8c9f110    158
6f2bebc01062eb8334dccba3e048fdb5    145
527ed53d2ba3a3bc417b8402d5b2f556    140
d7e500ad854a1b6ced39e53a525b8a6d    114
0737e4e4980f56c9fb1cb5743001c917    113
502db2fcfe26705ae16a46c5cb2ad2e5    112
715c9eb832dc833a0b6409ddccd268b1    107
f7456ac4e6b20911c40fdad18908a8d2    104
b4a21c82de4011033c8ac67081ff939c    104
b2be41464b53ffc6deae9536ddfd3aee    104
c3f4b3e345cb6b032db2e0459d179db3     98
0bfa3d952ffed50f25011b128e73a820     93
dbdfbfda2a4205bd59b22758ceddd5af     87
e4a512374eee079d2b8acc2ce69990d5     85
d39c2b0fb2e50e37795fdbe3b8cd3792     83
181962441153a36333f0c60701823412     78
e465fadd8b30e8669f397e32e10f6cd0     72
f2026b8cb48aff9af31577ecbfda5c38     71
8d7faa6d7f104a6cb7c4a9e1c6310a15     71
521896b01c1a506dc4404e600fa99c5b     69
37195f370c3fd7486ccedb1519b026c2     68
dba5f5266d03dd6d4db084ad7dbc683c     67
c85845b82e705a32f674757d8912df23     67
1695fbad7101e34d4cb4686a6770231f     66
eb7f8081aa0bd4004f513d3299db9063     65


### Our main column consists of the rows of sentences. One after another.
#### Make all words lower case

In [315]:
lines_of_combined_dataset = combined_dataset["STATUS"].str.lower()
print(lines_of_combined_dataset.head(n=5))

0    abcdefghijklmnopqrstuvwxyz qwertyuiopasdfghjkl...
1    feels aweful right now. why do you get sick li...
2    "i refuse to answer that question on the groun...
3    is enjoying the cricket...comfy boxers and rai...
4    life truly has special ways to show us what we...
Name: STATUS, dtype: object


### Now, let's split words for each sentence and make real sentences (python-usable objects) from rows.

In [316]:
# http://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize
twt = n.TreebankWordTokenizer()
english = n.data.load('tokenizers/punkt/PY3/english.pickle')

# Create lists
list_of_rows_CD = [l.split("\n")[0] for l in lines_of_combined_dataset]


list_of_splitted_words_CD, list_of_sentences = [], []

for k in list_of_rows_CD:
    list_of_sentences.append(n.sent_tokenize(k))
    
for kCD in list_of_rows_CD:
    list_of_splitted_words_CD.append(n.word_tokenize(kCD))

In [317]:
print(list_of_splitted_words_CD[:2], "\n\n", list_of_rows_CD[:2], "\n\n", list_of_sentences[:2])

[['abcdefghijklmnopqrstuvwxyz', 'qwertyuiopasdfghjklzxcvbnm', 'mnbvcxzlkjhgfdsapoiuytrewq', 'dea', 'dea', 'dea', ':', 'd', 'cartography+select', '.', 'topics', 'in', 'the', 'geography', 'of', 'china', '(', '?', '?', '?', '?', 'f', '?', 'add', '?', '...', ')', 'pa', '>', '<', 'dol', 'x', '2'], ['feels', 'aweful', 'right', 'now', '.', 'why', 'do', 'you', 'get', 'sick', 'like', 'this', 'on', 'weekends', 'only', '...', 'i', 'better', 'not', 'have', 'to', 'use', 'a', 'sick', 'day', 'tomorrow', 'or', 'i', 'will', 'be', 'pissed', '.', 'did', '*propname*', 'in', '16:37', ',', 'made', 'money', 'in', 'vegas', ',', 'and', 'just', 'had', 'his', '3rd', '10', 'hour', 'plus', 'day', 'this', 'month', '.', 'not', 'bad', '...', 'and', 'to', 'clarify', '*propname*', 'is', 'a', 'crossfit', 'workout', ',', '*propname*', 'get', 'your', 'head', 'out', 'of', 'the', 'gutter', 'did', '*propname*', 'in', '16:37', ',', 'made', 'money', 'in', 'vegas', ',', 'and', 'just', 'had', 'his', '3rd', '10', 'hour', 'plus', 

In [318]:
# https://stackoverflow.com/questions/2058985/python-count-sub-lists-in-nested-list
print("there are", sum(len(x) for x in list_of_splitted_words_CD), "tokens in", len(list_of_sentences), "sentences")

there are 117596 tokens in 242 sentences


In [319]:
print(list_of_rows_CD[:2], "\n")
print(list_of_sentences[:2],"\n") 
print(list_of_splitted_words_CD[:2], "\n")

['abcdefghijklmnopqrstuvwxyz qwertyuiopasdfghjklzxcvbnm mnbvcxzlkjhgfdsapoiuytrewq dea dea dea :d cartography+select. topics in the geography of china     (????f?add?...) pa><dol x 2', 'feels aweful right now. why do you get sick like this on weekends only... i better not have to use a sick day tomorrow or i will be pissed. did *propname* in 16:37, made money in vegas, and just had his 3rd 10 hour plus day this month. not bad...and to clarify *propname* is a crossfit workout, *propname* get your head out of the gutter did *propname* in 16:37, made money in vegas, and just had his 3rd 10 hour plus day this week. life is good did *propname* in 16:37, made money in vegas, and just had his 3rd 10 hour plus day this month. not bad did cindy 30 times in 20 minutes and gi jane in 12:11. save the sex joke, its crossfit. 2010 crossfit games here i come!!!!'] 

[['abcdefghijklmnopqrstuvwxyz qwertyuiopasdfghjklzxcvbnm mnbvcxzlkjhgfdsapoiuytrewq dea dea dea :d cartography+select.', 'topics in the 

### Different tokenizers (will decide later on which one)

Source: http://text-processing.com/demo/tokenize/


<img src="dif_tokenizers.png">

### Goal: Delete stopwords from each row

#### And store them again in each row (= nested list)

In [321]:
# stopwords.words("english") 
# https://stackoverflow.com/questions/19249201/how-to-create-a-number-of-empty-nested-lists-in-python
english_stops_list = list(set(stopwords.words('english')))

In [322]:
def delete_stopwords():
    english_stops = set(stopwords.words('english'))
    nest_list_without_stopwords = [[] for _ in range(len(list_of_splitted_words_CD))]
    for sentence in list_of_splitted_words_CD: 
        for word in sentence:
            if word not in english_stops:
                nest_list_without_stopwords[list_of_splitted_words_CD.index(sentence)].append(word)
    return nest_list_without_stopwords

In [323]:
nest_list_without_stopwords = delete_stopwords()

print(reprlib.repr(list_of_splitted_words_CD), "\n")
print(reprlib.repr(nest_list_without_stopwords))

[['abcdefghijklmnopqrstuvwxyz', 'qwertyuiopasdfghjklzxcvbnm', 'mnbvcxzlkjhgfdsapoiuytrewq', 'dea', 'dea', 'dea', ...], ['feels', 'aweful', 'right', 'now', '.', 'why', ...], ['``', 'i', 'refuse', 'to', 'answer', 'that', ...], ['is', 'enjoying', 'the', 'cricket', '...', 'comfy', ...], ['life', 'truly', 'has', 'special', 'ways', 'to', ...], ['just', 'booked', 'the', 'rest', 'of', 'my', ...], ...] 

[['abcdefghijklmnopqrstuvwxyz', 'qwertyuiopasdfghjklzxcvbnm', 'mnbvcxzlkjhgfdsapoiuytrewq', 'dea', 'dea', 'dea', ...], ['feels', 'aweful', 'right', '.', 'get', 'sick', ...], ['``', 'refuse', 'answer', 'question', 'grounds', "n't", ...], ['enjoying', 'cricket', '...', 'comfy', 'boxers', 'rainy', ...], ['life', 'truly', 'special', 'ways', 'show', 'us', ...], ['booked', 'rest', 'flights', '!', '!', '!', ...], ...]


### SnowballStemmer 

##### https://stackoverflow.com/questions/10554052/what-are-the-major-differences-and-benefits-of-porter-and-lancaster-stemming-alg

> Stemming is a technique to remove affixes from a word, ending up with the stem. For
example, the stem of cooking is cook, and a good stemming algorithm knows that the ing
suffix can be removed.


### Lemmatization 
is very similar to stemming, but is more akin to synonym replacement. A lemma is a root word, as opposed to the root stem. So unlike stemming, you are always left with a valid word that means the same thing. However, the word you end up with can be completely different.

### Compare different techniques

In [324]:
# Huge nested lists
# http://www.nltk.org/api/nltk.stem.html#module-nltk.stem.wordnet

nest_list_without_stopwords_ss = [[] for _ in range(len(nest_list_without_stopwords))]
nest_list_without_stopwords_pp = [[] for _ in range(len(nest_list_without_stopwords))]
nest_list_without_stopwords_ls = [[] for _ in range(len(nest_list_without_stopwords))]
nest_list_without_stopwords_lm = [[] for _ in range(len(nest_list_without_stopwords))]

ss = n.stem.SnowballStemmer("english")
pp = n.stem.PorterStemmer()
ls = n.stem.LancasterStemmer()
lm = n.stem.WordNetLemmatizer()

for sentence in nest_list_without_stopwords: 
    for word in sentence:
        nest_list_without_stopwords_ss[nest_list_without_stopwords.index(sentence)].append(ss.stem(word))
        nest_list_without_stopwords_pp[nest_list_without_stopwords.index(sentence)].append(pp.stem(word))
        nest_list_without_stopwords_lm[nest_list_without_stopwords.index(sentence)].append(lm.lemmatize(word, pos="v"))
        nest_list_without_stopwords_ls[nest_list_without_stopwords.index(sentence)].append(ls.stem(word))

print("Original \n", reprlib.repr(nest_list_without_stopwords[:6]), "\n")
print("SnowballStemmer \n", reprlib.repr(nest_list_without_stopwords_ss), "\n")
print("Porter \n", reprlib.repr(nest_list_without_stopwords_pp), "\n")
print("Lemmatizer \n", reprlib.repr(nest_list_without_stopwords_lm), "\n")
print("LancasterStemmer \n", reprlib.repr(nest_list_without_stopwords_ls), "\n")

Original 
 [['abcdefghijklmnopqrstuvwxyz', 'qwertyuiopasdfghjklzxcvbnm', 'mnbvcxzlkjhgfdsapoiuytrewq', 'dea', 'dea', 'dea', ...], ['feels', 'aweful', 'right', '.', 'get', 'sick', ...], ['``', 'refuse', 'answer', 'question', 'grounds', "n't", ...], ['enjoying', 'cricket', '...', 'comfy', 'boxers', 'rainy', ...], ['life', 'truly', 'special', 'ways', 'show', 'us', ...], ['booked', 'rest', 'flights', '!', '!', '!', ...]] 

SnowballStemmer 
 [['abcdefghijklmnopqrstuvwxyz', 'qwertyuiopasdfghjklzxcvbnm', 'mnbvcxzlkjhgfdsapoiuytrewq', 'dea', 'dea', 'dea', ...], ['feel', 'awe', 'right', '.', 'get', 'sick', ...], ['``', 'refus', 'answer', 'question', 'ground', "n't", ...], ['enjoy', 'cricket', '...', 'comfi', 'boxer', 'raini', ...], ['life', 'truli', 'special', 'way', 'show', 'us', ...], ['book', 'rest', 'flight', '!', '!', '!', ...], ...] 

Porter 
 [['abcdefghijklmnopqrstuvwxyz', 'qwertyuiopasdfghjklzxcvbnm', 'mnbvcxzlkjhgfdsapoiuytrewq', 'dea', 'dea', 'dea', ...], ['feel', 'awe', 'right', '.'

### Removing repeating characters

In [325]:
# book 38
class RepeatReplacer(object):
    def __init__(self):
        self.repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
        self.repl = r'\1\2\3'
        
    def replace(self, word):        
        if wordnet.synsets(word):
            return word
    
        repl_word = self.repeat_regexp.sub(self.repl, word)

        if repl_word != word:
            return self.replace(repl_word)
        else:
            return repl_word
        
    def delete_stupid_chars(self, word):
        """
        http://stackoverflow.com/a/3874768
        used above
        """
        replaced_word = self.replace(word)
        rem = "!?#.,();:'[].,//``...~<>$%^&*-_-=+"
        return replaced_word.translate(str.maketrans(dict.fromkeys(rem)))

replacer = RepeatReplacer()
replacer.delete_stupid_chars("!?sdf,word!??)()")

'sdfword'

In [326]:
def delete_repChars():
    nest_list_without_stopwords_lm_repchars = [[] for _ in range(len(nest_list_without_stopwords_lm))]
    for sentence in nest_list_without_stopwords_lm:
        for word in sentence:
            nest_list_without_stopwords_lm_repchars[nest_list_without_stopwords_lm.index(sentence)].append(replacer.delete_stupid_chars(word))
    return nest_list_without_stopwords_lm_repchars

In [327]:
nest_list_without_stopwords_lm_repchars = delete_repChars()
print(reprlib.repr(nest_list_without_stopwords_lm_repchars))

[['abcdefghijklmnopqrstuvwxyz', 'qwertyuiopasdfghjklzxcvbnm', 'mnbvcxzlkjhgfdsapoiuytrewq', 'dea', 'dea', 'dea', ...], ['feel', 'aweful', 'right', '', 'get', 'sick', ...], ['', 'refuse', 'answer', 'question', 'ground', 'nt', ...], ['enjoy', 'cricket', '', 'comfy', 'boxers', 'rainy', ...], ['life', 'truly', 'special', 'ways', 'show', 'us', ...], ['book', 'rest', 'flight', '', '', '', ...], ...]


In [328]:
#[nest_list_without_stopwords_lm_repchars for nest_list_without_stopwords_lm_repchars in nest_list_without_stopwords_lm_repchars if nest_list_without_stopwords_lm_repchars]
#    nest_list_without_stopwords_lm_repchars_as = [[] for _ in range(len(nest_list_without_stopwords_lm_repchars))]
#    for sentence in nest_list_without_stopwords_lm_repchars:
#        nest_list_without_stopwords_lm_repchars_as[nest_list_without_stopwords_lm_repchars.index(sentence)].append(list(filter(None, sentence)))   
#https://stackoverflow.com/questions/973568/convert-nested-lists-to-string

In [329]:
def delete_empty_strings():
    nest = [[] for _ in range(len(nest_list_without_stopwords_lm_repchars))]
    for sentence in nest_list_without_stopwords_lm_repchars: 
        for word in sentence:
            if word != '':
                nest[nest_list_without_stopwords_lm_repchars.index(sentence)].append(word)
    return nest

In [330]:
nest = delete_empty_strings()
outlst = [' '.join([str(c) for c in hm]) for hm in nest]

#print(outlst)
# print(len(nest_list_without_stopwords_lm_repchars[61]), "\n", len(nest_list_without_stopwords_lm[61]))

In [331]:
#http://aylien.com/web-summit-2015-tweets-part1
vectorizer = TfidfVectorizer(min_df=4, max_features = 10000)
vz = vectorizer.fit_transform(outlst)
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print("devil: " + str(tfidf["devil"]))

devil: 4.09516617054


### Create tagged words from all sentences

In [332]:
#print(reprlib.repr(nest_list_without_stopwords_lm))

print(list_of_rows_CD[0])
print(list_of_splitted_words_CD[0])
print(list_of_sentences[0], "\n")

tag_words = n.pos_tag(list_of_splitted_words_CD)
print(tag_words, "\n")
# print(reprlib.repr(tagged_words()))


abcdefghijklmnopqrstuvwxyz qwertyuiopasdfghjklzxcvbnm mnbvcxzlkjhgfdsapoiuytrewq dea dea dea :d cartography+select. topics in the geography of china     (????f?add?...) pa><dol x 2
['abcdefghijklmnopqrstuvwxyz', 'qwertyuiopasdfghjklzxcvbnm', 'mnbvcxzlkjhgfdsapoiuytrewq', 'dea', 'dea', 'dea', ':', 'd', 'cartography+select', '.', 'topics', 'in', 'the', 'geography', 'of', 'china', '(', '?', '?', '?', '?', 'f', '?', 'add', '?', '...', ')', 'pa', '>', '<', 'dol', 'x', '2']
['abcdefghijklmnopqrstuvwxyz qwertyuiopasdfghjklzxcvbnm mnbvcxzlkjhgfdsapoiuytrewq dea dea dea :d cartography+select.', 'topics in the geography of china     (????f?add?...)', 'pa><dol x 2'] 



AttributeError: 'list' object has no attribute 'isdigit'

In [333]:
nest_list_tagged_words = [[] for _ in range(len(nest_list_without_stopwords_lm_repchars))]

#def tagged_words():
#    for sentence in nest_list_without_stopwords_lm_repchars:
#        for words in sentence:
#            nest_list_tagged_words[nest_list_without_stopwords_lm_repchars.index(sentence)].append(n.pos_tag(words))
#    return nest_list_tagged_words


### Unigram features

Use -a for code analysis

In [334]:
def word_fea(words):
    return dict((word, True))
word_fea(outlst)

ValueError: dictionary update sequence element #0 has length 9; 2 is required

In [335]:
# Bigram collocation
# https://github.com/neotenic/cancer/blob/master/nltk.ipynb
def bigram_features(words, score_fn=BAM.chi_sq): 
    bg_finder = BigramCollocationFinder.from_words(words) 
    bigrams = bg_finder.nbest(score_fn, 100000) 
    return dict((bg, True) for bg in chain(words, bigrams)) 

#bigram_features(outlst, score_fn=BAM.chi_sq)

In [336]:
classifier = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC()))])

In [337]:
cl = n.NaiveBayesClassifier.train(train)
print(n.classify.accuracy(cl, test),"%.3f")
cl.show_most_informative_features(40)
cl.prob_classify(featurize(name)) #

ValueError: too many values to unpack (expected 2)